In [1]:
import os
import sys
import pickle
import datetime
import numpy as np
import pandas as pd
from example_lorenz import get_lorenz_data

sys.path.append("../")
from training import train_network
from autoencoder import library_size

## Generating the Training and Validation data

In [2]:
noise           = 1e-6
training_data   = get_lorenz_data(16, noise)
validation_data = get_lorenz_data(2, noise)

# Save the data
data_to_save = {
    "training_data": training_data,
    "validation_data": validation_data
}
with open("lorenz_data.pkl", "wb") as f:
    pickle.dump(data_to_save, f)

## Set up model and training parameters

In [5]:
# Retrieve the data
with open("lorenz_data.pkl", "rb") as f:
    data = pickle.load(f)

training_data                                       = data["training_data"]
validation_data                                     = data["validation_data"]

params = {}

params['input_dim']                                 = 128
params['latent_dim']                                = 3
params['model_order']                               = 1
params['poly_order']                                = 3
params['include_sine']                              = False
params['library_dim']                               = library_size(params['latent_dim'], params['poly_order'], params['include_sine'], True)

# sequential thresholding parameters
params['sequential_thresholding']                   = True
params['coefficient_threshold']                     = 0.1
params['threshold_frequency']                       = 500
params['coefficient_mask']                          = np.ones((params['library_dim'], params['latent_dim']))
params['coefficient_initialization']                = 'constant'

# loss function weighting
params['loss_weight_decoder']                       = 1.0
params['loss_weight_sindy_z']                       = 0.0
params['loss_weight_sindy_x']                       = 1e-4
params['loss_weight_sindy_regularization']          = 1e-5

params['activation']                                = 'sigmoid'
params['widths']                                    = [64,32]

# training parameters
params['epoch_size']                                = training_data['x'].shape[0]
params['batch_size']                                = 100
params['learning_rate']                             = 1e-3

params['data_path']                                 = os.getcwd() + '/'
params['print_progress']                            = True
params['print_frequency']                           = 100

# training time cutoffs
params['max_epochs']                                = 2
params['refinement_epochs']                         = 1001

## Run training experiment

In [6]:
n_experiments = 1 
df = pd.DataFrame()

for i in range(n_experiments):
    params['coefficient_mask'] = np.ones((params['library_dim'], params['latent_dim']))
    params['save_name'] = 'lorenz_' + datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S_%f")

    results_dict = train_network(training_data, validation_data, params)
    print(1)
    df = df.append({**results_dict, **params}, ignore_index=True)

df.to_pickle('experiment_results_' + datetime.datetime.now().strftime("%Y%m%d%H%M") + '.pkl')

TypeError: mean(): argument 'input' (position 1) must be Tensor, not numpy.ndarray